In [26]:
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_core.tools import tool
import getpass
import os
prompt = ChatPromptTemplate.from_template(
    """
    Gevin this is the user question "{query}" \
    and this the answer "{answer}"\
    Act as a virual assistant and repsond like human providing the answer for the user
    """)

chain = LLMChain(llm=llm, prompt=prompt)


# os.environ["GROQ_API_KEY"] = getpass.getpass()


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return chain.run(query="what is 2 + 2", answer="4")


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b

@tool
def greating(query: str) -> str:
    """Welcome the user prompt by discriping the serivce that you provide.

    Args:
        query: the user prompt
    """
    return "This Sensorconnect framework"


tools = [add, multiply,greating]

In [27]:

llm = ChatGroq(model="llama3-8b-8192")
llm_with_tools = llm.bind_tools(tools)

# query = "Hello"
query="what is 2 + 2"
AIMessage=llm_with_tools.invoke(query)
AIMessage

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2nph', 'function': {'arguments': '{"a":2,"b":2}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 1151, 'total_tokens': 1230, 'completion_time': 0.063500178, 'prompt_time': 0.17313925, 'queue_time': None, 'total_time': 0.23663942799999999}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0e194062-787c-4233-89de-bccf4247a188-0', tool_calls=[{'name': 'add', 'args': {'a': 2, 'b': 2}, 'id': 'call_2nph', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1151, 'output_tokens': 79, 'total_tokens': 1230})

In [28]:
Ai_call=AIMessage.tool_calls
for i in Ai_call:
    selected_tool = {"add": add, "multiply": multiply, "greating":greating}[i["name"].lower()]
    print(f"selected_tool: {selected_tool}")
    tool_msg = selected_tool.invoke(i)
    tool_msg

selected_tool: name='add' description='Adds a and b.' args_schema=<class 'pydantic.main.addSchema'> func=<function add at 0x000002B99FA7F4C0>


In [29]:
from langchain_core.messages import HumanMessage
query = "What is 3 * 12? Also, what is 11 + 49?"
messages = [HumanMessage(query)]
messages.append(tool_msg)

In [30]:
messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?'),
 ToolMessage(content="You're wondering what 2 + 2 is? Well, let me think about that for a sec... Ah-ha! I've got it! The answer is... 4! That's right, when you add 2 and 2 together, you get 4!", name='add', tool_call_id='call_2nph')]

In [19]:
prompt_formatted_str

'Human: \n    Gevin this is the user query: what is 2 + 2     and this the answer 4"    Act as a virual assistant and repsond like human providing the answer for the user\n    '